In [1]:
import urllib.request
import urllib.parse
import re
import pandas as pd
pd.options.display.max_rows = 2000

In [2]:
tracks = pd.read_csv('titles.csv')

In [3]:
# cleaning artists
tracks['artists'] = tracks['artists'].apply\
    (lambda x: x[len('\ufeff'):] if x[:len('\ufeff')] == '\ufeff' else x)

In [4]:
# how many distinct artists do I have?
artists = set()
for a in list(tracks['artists'].unique()):
#     print('-------', a)
    names = a.split(' ')
    for n in names:
        artists.add(n)
len(artists)

829

In [5]:
# to check possible typos
sorted_artists = list(artists)
sorted_artists.sort()
# sorted_artists

In [6]:
# what are the most frequent artists?
artist_occurrences = dict()
for a in list(tracks['artists']):
    names = a.split(' ')
    for n in names:
        if n not in artist_occurrences.keys():
            artist_occurrences[n] = 0
        artist_occurrences[n] += 1

In [7]:
artist_occurrences_list = []
for a, o in artist_occurrences.items():
#     print('-',o)
    artist_occurrences_list.append([a, o])
artist_occurrences_list.sort(key=lambda x: -x[1])

In [8]:
artist_occurrences_list[:20]

[['Paul_Kalkbrenner', 44],
 ['Ludovico_Einaudi', 43],
 ['Yann_Tiersen', 40],
 ['Coldplay', 37],
 ['Flume', 29],
 ['Enrico_Caruso', 27],
 ['Chopin', 24],
 ['Fritz_Kalkbrenner', 24],
 ['Nils_Frahm', 21],
 ['Florence_and_The_Machine', 20],
 ['Daughter', 19],
 ['Fabri_Fibra', 19],
 ['Beriot', 18],
 ['Hardwell', 18],
 ['David_Guetta', 17],
 ['Johann_Strauss_II', 16],
 ['Benny_Benassi', 16],
 ['Arvo_Pärt', 16],
 ['Locatelli', 15],
 ['The_xx', 15]]

# listen to tracks

In [9]:
tracks_list = tracks.values.tolist()

In [10]:
def printt(s):
    print('  ' + str(s))


In [11]:
import numpy as np
import math

In [50]:
i -= 1
# i = 1397
# i = len(tracks_list) - 1
t = tracks_list[i]
search = ''
for el in t:
    try:
        if math.isnan(el):
            pass
#             printt(el)
    except:
        el = el.replace('_', ' ')
        search += (el) + ' '
search = search[:-1]
query_string = urllib.parse.urlencode({"search_query" : search})
html_content = urllib.request.urlopen("http://www.youtube.com/results?" + query_string)
search_results = re.findall(r'href=\"\/watch\?v=(.{11})', html_content.read().decode())
print(search)
print("http://www.youtube.com/watch?v=" + search_results[0])

# to get genre and bpm
sb = search + ' ' 'beatport'
sbl = sb.split(' ')
s2 = '+'.join(sbl)
print("https://www.google.com/search?q=" + s2)


Recondite Theater II
http://www.youtube.com/watch?v=SxCxgDdehxo
https://www.google.com/search?q=Recondite+Theater+II+beatport


In [79]:
[s + '+' for s in search.split(' ')] + ['beatport']

['Steve+', 'Parker+', 'The+', 'Prophet+', 'beatport']

artist,title,genre,bpm,src,tags

tags: soft, chill, heavy, piano, voice, synth, brass, strings, orchestra